In [1]:
import pandas as pd

from src.common import *
import src.mongos as mg

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

# Load

In [2]:
raw = mg.read_mongo('electoral_system', 'systems', )
raw.head()

,Code,Compulsory/voluntary voting,Country,Electoral Management model,Electoral System (Chamber 1),President,Voting age,Voting outside the country is permitted for
0,AF,Voting is voluntary,Afghanistan,Independent,Single Non-Transferable Vote,Yes,18,"Other, No information available"
1,AL,Voting is voluntary,Albania,Independent,List Proportional Representation,Yes,18,Not applicable
2,DZ,Voting is voluntary,Algeria,Governmental,List Proportional Representation,Yes,18,Citizens residing outside the country
3,AS,Voting is voluntary,American Samoa,Mixed,Plurality (FPTP),Yes,18,"Citizens outside the country (including those on vacation), Members of the armed forces, Students, Diplomatic staff"
4,AD,Voting is voluntary,Andorra,Mixed,Parallel (Segmented) (PR Lists and Majoritarian constituencies),No,18,"Citizens residing outside the country, Students, Other"


# Explore

## Info

In [3]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 8 columns):
Code                                           236 non-null object
Compulsory/voluntary voting                    236 non-null object
Country                                        236 non-null object
Electoral Management model                     236 non-null object
Electoral System (Chamber 1)                   236 non-null object
President                                      236 non-null object
Voting age                                     236 non-null object
Voting outside the country is permitted for    236 non-null object
dtypes: object(8)
memory usage: 14.8+ KB


## Describe

In [4]:
raw.describe()

,Code,Compulsory/voluntary voting,Country,Electoral Management model,Electoral System (Chamber 1),President,Voting age,Voting outside the country is permitted for
count,236,236,236,236,236,236,236,236
unique,236,9,236,6,25,5,10,35
top,AE,Voting is voluntary,Somalia,Independent,List Proportional Representation,Yes,18,Citizens residing outside the country
freq,1,193,1,141,76,164,203,59


## nan?

In [5]:
raw.isna().any()

Code                                           False
Compulsory/voluntary voting                    False
Country                                        False
Electoral Management model                     False
Electoral System (Chamber 1)                   False
President                                      False
Voting age                                     False
Voting outside the country is permitted for    False
dtype: bool

## empty string

In [6]:
raw.applymap(lambda x: x == '').any()

Code                                           False
Compulsory/voluntary voting                    True 
Country                                        False
Electoral Management model                     True 
Electoral System (Chamber 1)                   True 
President                                      True 
Voting age                                     True 
Voting outside the country is permitted for    True 
dtype: bool

# Transform

## Pre-process

In [7]:
def pre_process(raw):
    system = raw.copy()
    system.rename(columns={
            'Compulsory/voluntary voting': 'Voluntary',
            'Electoral Management model': 'Model',
            'Electoral System (Chamber 1)':'System',
            'Voting age': 'Age',
            'Voting outside the country is permitted for': 'Outside',
        },
        inplace=True
    )
    return system

system = pre_process(raw)
system.head()

,Code,Voluntary,Country,Model,System,President,Age,Outside
0,AF,Voting is voluntary,Afghanistan,Independent,Single Non-Transferable Vote,Yes,18,"Other, No information available"
1,AL,Voting is voluntary,Albania,Independent,List Proportional Representation,Yes,18,Not applicable
2,DZ,Voting is voluntary,Algeria,Governmental,List Proportional Representation,Yes,18,Citizens residing outside the country
3,AS,Voting is voluntary,American Samoa,Mixed,Plurality (FPTP),Yes,18,"Citizens outside the country (including those on vacation), Members of the armed forces, Students, Diplomatic staff"
4,AD,Voting is voluntary,Andorra,Mixed,Parallel (Segmented) (PR Lists and Majoritarian constituencies),No,18,"Citizens residing outside the country, Students, Other"


## Index

In [15]:
idx_numeric_age = system['Age'].str.isnumeric()


## Columns

# Questions

## Voting Age

In [9]:
system['Age'].value_counts()

18                          203
16                          9  
21                          7  
20                          4  
17                          3  
Other (specify)             3  
No information available    3  
Not applicable              2  
19                          1  
                            1  
Name: Age, dtype: int64

### non-numeric age

In [17]:
system.loc[~idx_numeric_age]

,Code,Voluntary,Country,Model,System,President,Age,Outside
28,BN,Not applicable,Brunei Darussalam,Not applicable,Not applicable,No,Not applicable,Not applicable
91,VA,No information available,Holy See (Vatican City State),Not applicable,Not applicable,Yes,Other (specify),No information available
102,IT,Voting is voluntary,Italy,Governmental,List Proportional Representation,Yes,Other (specify),Citizens residing outside the country
185,SA,Not applicable,Saudi Arabia,Not applicable,Not applicable,No,Not applicable,Not applicable
194,SO,Voting is voluntary,Somalia,Independent,Not applicable,Yes,No information available,No information available
210,TK,,Tokelau,Mixed,Two-Round System,No,,
220,AE,Voting is voluntary,United Arab Emirates,Governmental,Block Vote,Yes,Other (specify),No information available
230,WF,Voting is voluntary,Wallis and Futuna,,Two-Round System,No information available,No information available,Citizens residing outside the country
231,EH,No information available,Western Sahara,,No information available,No information available,No information available,No information available


### 19: only korea

In [11]:
system.loc[system['Age']=='19']

,Code,Voluntary,Country,Model,System,President,Age,Outside
111,KR,Voting is voluntary,"Korea, Republic of",Independent,Parallel (Segmented) (PR Lists and Majoritarian constituencies),Yes,19,Citizens outside the country (including those on vacation)


### 20: 4 countries

In [16]:
system.loc[system['Age']=='20']

,Code,Voluntary,Country,Model,System,President,Age,Outside
15,BH,Voting is voluntary,Bahrain,Governmental,Two-Round System,No,20,Citizens outside the country (including those on vacation)
34,CM,Voting is voluntary,Cameroon,Independent,Parallel (Segmented) (PR Lists and Majoritarian constituencies),Yes,20,Citizens residing outside the country
147,NR,Voting is compulsory and regulated in the electoral law,Nauru,Governmental,Alternative Vote,Yes,20,"Citizens residing outside the country, Citizens outside the country (including those on vacation)"
204,TW,Voting is voluntary,Taiwan,Independent,Parallel (Segmented) (PR Lists and Majoritarian constituencies),Yes,20,Outside the country voting is not permitted


### 21: 7 countries

In [12]:
system.loc[system['Age']=='21']

,Code,Voluntary,Country,Model,System,President,Age,Outside
112,KW,Voting is voluntary,Kuwait,Governmental,Block Vote,No,21,Outside the country voting is not permitted
116,LB,Voting is voluntary,Lebanon,Governmental,"Block Vote, List Proportional Representation",Yes,21,Citizens residing outside the country
127,MY,Voting is voluntary,Malaysia,Independent,Plurality (FPTP),No,21,"Members of the armed forces, Students, Diplomatic staff"
160,OM,Voting is voluntary,Oman,Independent,Plurality (FPTP),No,21,Outside the country voting is not permitted
182,WS,Voting is voluntary,Samoa,Independent,Plurality (FPTP),Not applicable,21,Outside the country voting is not permitted
190,SG,Voting is compulsory and regulated in the electoral law,Singapore,Governmental,"Plurality (FPTP), Other",Yes,21,"Citizens outside the country (including those on vacation), Members of the armed forces, Students, Diplomatic staff, Other"
211,TO,Voting is voluntary,Tonga,Independent,Plurality (FPTP),No,21,Outside the country voting is not permitted


## System

In [18]:
system['System'].value_counts()

List Proportional Representation                                                                                       76
Plurality (FPTP)                                                                                                       54
Parallel (Segmented) (PR Lists and Majoritarian constituencies)                                                        26
Two-Round System                                                                                                       16
Block Vote                                                                                                             9 
No information available                                                                                               8 
Mixed Member Proportional                                                                                              7 
Not applicable                                                                                                         6 
                        